In [1]:
import pandas as pd
df = pd.read_csv("products_train.csv")
df.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551057 entries, 0 to 1551056
Data columns (total 11 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   id        1551057 non-null  object 
 1   locale    1551057 non-null  object 
 2   title     1551049 non-null  object 
 3   price     1551057 non-null  float64
 4   brand     1531691 non-null  object 
 5   color     1125432 non-null  object 
 6   size      917091 non-null   object 
 7   model     761137 non-null   object 
 8   material  834382 non-null   object 
 9   author    73509 non-null    object 
 10  desc      1424083 non-null  object 
dtypes: float64(1), object(10)
memory usage: 130.2+ MB
None


In [3]:
print(df.describe())

              price
count  1.551057e+06
mean   1.806107e+06
std    8.302930e+06
min    0.000000e+00
25%    1.119000e+01
50%    2.328000e+01
75%    7.990000e+02
max    4.000000e+07


In [4]:
uk_samples = df[df['locale'] == 'UK'].copy()

# Print the extracted samples
uk_samples.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
913336,B087LZNPHS,UK,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid...",24.99,SOCHOW,Teal Green,127cm×150cm,NaN,100% Polyester,NaN,COLOR: The sherpa throw blanket is available i...
913337,B08THFN1KX,UK,Hippowarehouse Personalised Photo Printed Mous...,9.95,Hippowarehouse,White,240mm x 190mm x 60mm,50245-Mat-Perso,Rubber,NaN,Competitively priced
913338,0804185328,UK,"500 Easy Recipes for Every Machine, Both Stove...",16.49,Clarkson Potter,White,NaN,NaN,NaN,"Scarbrough, Mark",NaN
913339,B09VBKDBW6,UK,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S...",11.99,TYHJOY,Black,NaN,FBA-sealer-black,Acrylonitrile Butadiene Styrene,NaN,【AFTER-SALE】This handheld food heat sealer sho...
913340,B096ZW8B49,UK,Lucosobie Steering Wheel Lock - Car Anti-Theft...,26.99,Lucosobie,Black,NaN,NaN,Alloy Steel,NaN,🔐【 Anti-Friction & Customer First】Each box of ...


In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import re

# # Download required resources from NLTK
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

# Define the preprocessing functions
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    if isinstance(text, str):
        # Convert text to lowercase
        text = text.lower()
        
        # Remove brackets and non-alphabetic characters
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
        
        # Sentence tokenization
        sentences = sent_tokenize(cleaned_text)

        # Word tokenization, Stemming, Lemmatization, and Stop Word Removal
        preprocessed_words = []
        for sentence in sentences:
            words = word_tokenize(sentence)
            stemmed_words = [stemmer.stem(word) for word in words]
            lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
            filtered_words = [word for word in lemmatized_words if word.lower() not in stop_words]
            preprocessed_words.extend(filtered_words)

        return preprocessed_words
    else:
        return []

# Apply the text preprocessing pipeline to the 'desc' column
uk_samples['desc_processed'] = uk_samples['desc'].apply(lambda x: preprocess(x))
uk_samples['title_processed'] = uk_samples['title'].apply(lambda x: preprocess(x))

# Create a new column that combines the words from 'desc_processed' and 'title_processed'
uk_samples['combined_processed'] = uk_samples['desc_processed'] + uk_samples['title_processed']

# Print the updated DataFram
uk_samples.head()

,id,locale,title,price,brand,color,size,model,material,author,desc,desc_processed,title_processed,combined_processed
913336,B087LZNPHS,UK,"SOCHOW Sherpa Fleece Throw Blanket, Double-Sid...",24.99,SOCHOW,Teal Green,127cm×150cm,NaN,100% Polyester,NaN,COLOR: The sherpa throw blanket is available i...,"[color, sherpa, throw, blanket, avail, varieti...","[sochow, sherpa, fleec, throw, blanket, double...","[color, sherpa, throw, blanket, avail, varieti..."
913337,B08THFN1KX,UK,Hippowarehouse Personalised Photo Printed Mous...,9.95,Hippowarehouse,White,240mm x 190mm x 60mm,50245-Mat-Perso,Rubber,NaN,Competitively priced,"[competit, price]","[hippowarehous, personalis, photo, print, mous...","[competit, price, hippowarehous, personalis, p..."
913338,0804185328,UK,"500 Easy Recipes for Every Machine, Both Stove...",16.49,Clarkson Potter,White,NaN,NaN,NaN,"Scarbrough, Mark",NaN,[],"[easi, recip, everi, machin, stovetop, electr,...","[easi, recip, everi, machin, stovetop, electr,..."
913339,B09VBKDBW6,UK,"TYHJOY Mini Bag Sealer, Handheld Vacuum Heat S...",11.99,TYHJOY,Black,NaN,FBA-sealer-black,Acrylonitrile Butadiene Styrene,NaN,【AFTER-SALE】This handheld food heat sealer sho...,"[aftersalethi, handheld, food, heat, sealer, p...","[tyhjoy, mini, bag, sealer, handheld, vacuum, ...","[aftersalethi, handheld, food, heat, sealer, p..."
913340,B096ZW8B49,UK,Lucosobie Steering Wheel Lock - Car Anti-Theft...,26.99,Lucosobie,Black,NaN,NaN,Alloy Steel,NaN,🔐【 Anti-Friction & Customer First】Each box of ...,"[antifrict, custom, firsteach, box, product, e...","[lucosobi, steer, wheel, lock, car, antitheft,...","[antifrict, custom, firsteach, box, product, e..."


In [ ]:
words = uk_samples[uk_samples['id'] == 'B087LZNPHS']['combined_processed']

for word in words:
    print(word)


In [ ]:
words = uk_samples[uk_samples['id'] == 'B087LZNPHS']['unique_words']

for word in words:
    print(word)


In [6]:
df_session = pd.read_csv("sessions_train.csv")
uk_df = df_session[df_session['locale'] == 'UK'].copy()
uk_df.head()

,prev_items,next_item,locale
2090535,['B0BFDL54Y7' 'B0BFDR9X13' 'B07J4WF8VH' 'B07Y2...,B07Y227WNJ,UK
2090536,['B07FM2GLNQ' 'B07GZW3P4W'],B095NNZCR6,UK
2090537,['B0021L95HU' 'B07DDL77RY' 'B07DDL77RY'],B002KA1FZC,UK
2090538,['B0B2WSZYL2' 'B000I8XZ7O'],B000I90TAO,UK
2090539,['B00LW1APOC' 'B00LW1APOC' 'B00OI6NQUI' 'B09HL...,B07H54NZ3K,UK


In [7]:
def extract_ids(ids_str):
    ids_str = ids_str.replace("[", "").replace("]", "").replace("'", "")
    return [id_.strip() for id_ in ids_str.split(" ") if id_.strip() != ' ']



# Apply the function to each element of the 'prev_items' column and assign the result to a new column 'prev_item_ids'
uk_df['prev_item_ids'] = uk_df['prev_items'].apply(extract_ids)

In [8]:
# Step 1: Get the unique words from uk_samples
unique_words_dict = dict(zip(uk_samples['id'], uk_samples['combined_processed']))

# Step 2: Define a function to retrieve unique words for a single item ID
def get_unique_words(item_id):
    unique_words = []
    if item_id in unique_words_dict:
        unique_words = unique_words_dict[item_id]
    return unique_words

# Step 3: Apply the function to the 'prev_item_ids' column in uk_df
uk_df['prev_item_unique_words'] = uk_df['prev_item_ids'].apply(lambda x: [word for item_id in x for word in get_unique_words(item_id)])
# Step 4: Apply the function to the 'next_item' column in uk_df
uk_df['next_item_unique_words'] = uk_df['next_item'].apply(lambda x: get_unique_words(x) if x in unique_words_dict else [])


# Step 5: Join the words into sentences
uk_df['prev_sentence'] = uk_df['prev_item_unique_words'].apply(lambda x: ' '.join(x))
uk_df['next_sentence'] = uk_df['next_item_unique_words'].apply(lambda x: ' '.join(x))

# Display the updated DataFrame
uk_df.head()


,prev_items,next_item,locale,prev_item_ids,prev_item_unique_words,next_item_unique_words,prev_sentence,next_sentence
2090535,['B0BFDL54Y7' 'B0BFDR9X13' 'B07J4WF8VH' 'B07Y2...,B07Y227WNJ,UK,"[B0BFDL54Y7, B0BFDR9X13, B07J4WF8VH, B07Y21LDJX]","[easi, cleanth, blade, easi, remov, easi, clea...","[easi, clean, adov, veget, slicer, complet, di...",easi cleanth blade easi remov easi clean thoro...,easi clean adov veget slicer complet disassemb...
2090536,['B07FM2GLNQ' 'B07GZW3P4W'],B095NNZCR6,UK,"[B07FM2GLNQ, B07GZW3P4W]","[accur, control, ergonom, dont, bore, thi, pc,...","[adopt, pure, plain, color, voil, curtain, dro...",accur control ergonom dont bore thi pc mous ac...,adopt pure plain color voil curtain drop pair ...
2090537,['B0021L95HU' 'B07DDL77RY' 'B07DDL77RY'],B002KA1FZC,UK,"[B0021L95HU, B07DDL77RY, B07DDL77RY]","[conveni, dosag, seven, sea, cod, liver, oil, ...","[contain, natur, sourc, omega, fish, oil, cod,...",conveni dosag seven sea cod liver oil tablet p...,contain natur sourc omega fish oil cod liver o...
2090538,['B0B2WSZYL2' 'B000I8XZ7O'],B000I90TAO,UK,"[B0B2WSZYL2, B000I8XZ7O]","[suit, az, ax, hv, hv, hv, nv, nv, hv, nv, nv,...","[includ, hood, jumpsuit, rubi, offici, hallowe...",suit az ax hv hv hv nv nv hv nv nv nv nv nv nv...,includ hood jumpsuit rubi offici halloween hau...
2090539,['B00LW1APOC' 'B00LW1APOC' 'B00OI6NQUI' 'B09HL...,B07H54NZ3K,UK,"[B00LW1APOC, B00LW1APOC, B00OI6NQUI, B09HLDN8W1]","[damagefre, decor, say, goodby, hole, sticki, ...","[damag, free, hang, damag, free, hang, strip, ...",damagefre decor say goodby hole sticki residu ...,damag free hang damag free hang strip work wit...


In [10]:
# from gensim.models import Word2Vec
# import numpy as np

# # Train Word2Vec model on the preprocessed 'desc' data
# model = Word2Vec(sentences=uk_df['prev_sentence'], vector_size=100, window=5, min_count=1, workers=4)

# # Function to calculate the word embedding vector for a specific product
# def calculate_product_embedding(desc):

#     if isinstance(desc, str):
        
#         desc_embedding = np.zeros(model.vector_size)
#         num_desc_words = 0
#         for word in desc.split():
#             if word in model.wv:
#                 desc_embedding += model.wv[word]
#                 num_desc_words += 1

#         if num_desc_words > 0:
#             desc_embedding /= num_desc_words
#             return desc_embedding
    
#     return None



# uk_df['X_embeddings'] = uk_df.apply(lambda row: calculate_product_embedding(row['prev_sentence']), axis=1)
# uk_df['Y_embeddings'] = uk_df.apply(lambda row: calculate_product_embedding(row['next_sentence']), axis=1)

# uk_df.head()

In [9]:
sample_size = 10000
sample = uk_df.sample(n=sample_size, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the preprocessed 'desc' data
vectorizer.fit(uk_df['prev_sentence'])

# Function to calculate the TF-IDF embedding vector for a specific product
def calculate_product_embedding(desc):
    if isinstance(desc, str):
        desc_embedding = vectorizer.transform([desc]).toarray()[0]
        return desc_embedding
    return None

sample['X_embeddings'] = sample.apply(lambda row: calculate_product_embedding(row['prev_sentence']), axis=1)
sample['Y_embeddings'] = sample.apply(lambda row: calculate_product_embedding(row['next_sentence']), axis=1)

sample.head()

MemoryError: 

In [ ]:
uk_df.shape

In [ ]:
print(uk_samples.info())

In [ ]:
print(uk_df.info())

In [ ]:
uk_df['X_embeddings'].dropna()
uk_df['Y_embeddings'].dropna()

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Extract the input and target values
X = np.stack(sample['X_embeddings'].values)
y = np.stack(sample['Y_embeddings'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Step 3: Define and train the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train.shape[1]))
# Output layer with the same dimension as y_train
learning_rate = 0.01  # Set the desired learning rate
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, batch_size=32)
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, batch_size=32)

# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


KeyError: 'X_embeddings'

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Extract the input and target values
X = np.stack(uk_df['X_embeddings'].values)
y = np.stack(uk_df['Y_embeddings'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Step 3: Define and train the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(y_train.shape[1]))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from keras.regularizers import l2

# Extract the input and target values
X = np.stack(sample['X_embeddings'].values)
y = np.stack(sample['Y_embeddings'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Step 3: Define and train the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, batch_size=32)

# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# Extract the input and target values
X = np.stack(sample['X_embeddings'].values)
y = np.stack(sample['Y_embeddings'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Step 3: Define and train the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(128), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(100, activation='softmax')) 

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, batch_size=32)

# Step 4: Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
